In [3]:
from datetime import datetime as dt
import pandas as pd
import time
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [4]:
# url link from - https://www.redfin.com/news/data-center/
url_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

def extract_data(url):
    df = pd.read_csv(url, compression='gzip', sep='\t')    
    return df
    
df = extract_data(url_by_city)
df.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,median_ppsf_yoy,median_list_ppsf,median_list_ppsf_mom,median_list_ppsf_yoy,homes_sold,homes_sold_mom,homes_sold_yoy,pending_sales,pending_sales_mom,pending_sales_yoy,new_listings,new_listings_mom,new_listings_yoy,inventory,inventory_mom,inventory_yoy,months_of_supply,months_of_supply_mom,months_of_supply_yoy,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2012-01-01,2012-01-31,30,place,6,4183,f,"Coplay, PA",Coplay,Pennsylvania,PA,Single Family Residential,6,117000.0,-0.387435,NaN,129900.0,NaN,NaN,97.500000,-0.151821,NaN,89.627358,NaN,NaN,1.0,-0.500000,NaN,1.0,-0.500000,NaN,4.0,NaN,NaN,14.0,0.166667,NaN,14.0,8.0,NaN,601.0,448.0,NaN,0.975813,0.014940,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,"Allentown, PA",10900,2024-11-12 14:29:49
1,2022-03-01,2022-03-31,30,place,6,15196,f,"Pen Argyl, PA",Pen Argyl,Pennsylvania,PA,Single Family Residential,6,118750.0,-0.403266,-0.090038,174450.0,0.246962,0.151865,90.510671,-0.195865,-0.123894,112.242123,-0.191279,0.069219,1.0,-0.666667,-0.750000,3.0,-0.250000,-0.250000,3.0,0.0,-0.571429,2.0,-0.500000,-0.500000,2.0,0.7,1.0,80.0,57.0,-5.0,1.136364,0.164327,0.154224,1.000000,0.666667,0.750000,1.000000,0.50000,0.75000,0.000000,-0.250000,-0.750000,"Allentown, PA",10900,2024-11-12 14:29:49
2,2018-12-01,2018-12-31,30,place,6,6505,f,"Evans, CO",Evans,Colorado,CO,All Residential,-1,299000.0,0.049123,0.128302,285000.0,-0.021459,0.075472,134.523810,-0.053560,0.110963,127.691195,-0.097681,-0.099146,29.0,0.000000,-0.355556,29.0,0.074074,-0.216216,14.0,-0.5,-0.517241,42.0,-0.263158,-0.322581,1.4,-0.6,0.0,42.0,6.0,4.0,0.987725,-0.011298,-0.011388,0.137931,-0.206897,-0.239847,0.166667,-0.04386,0.11828,0.413793,-0.104725,-0.180801,"Greeley, CO",24540,2024-11-12 14:29:49
3,2015-12-01,2015-12-31,30,place,6,24247,f,"Mentone, CA",Mentone,California,CA,Condo/Co-op,3,135000.0,0.080000,NaN,NaN,NaN,NaN,143.008475,0.080000,NaN,NaN,NaN,NaN,1.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,-2.0,NaN,1.000000,0.074074,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Riverside, CA",40140,2024-11-12 14:29:49
4,2012-12-01,2012-12-31,30,place,6,10465,f,"Lakewood, NY",Lakewood,New York,NY,All Residential,-1,106250.0,0.062527,-0.440789,87100.0,-0.497548,1.182957,73.199745,0.147657,-0.078420,77.961152,-0.375089,1.543995,4.0,-0.333333,-0.200000,4.0,-0.333333,-0.200000,4.0,0.0,3.000000,43.0,-0.044444,0.433333,10.8,3.3,4.8,157.0,-59.0,8.0,0.937659,-0.003957,-0.037512,0.000000,-0.166667,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,"Jamestown, NY",27460,2024-11-12 14:29:49


In [5]:
print('Num of rows:', len(df))
print('Num of cols:', len(df.columns))

Num of rows: 5637368
Num of cols: 58


In [7]:
# url link from - https://www.redfin.com/news/data-center/
url_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

def lambda_handler(url):
    df = pd.read_csv(url, compression='gzip', sep='\t')
   
   
     # Remove commas from the 'city' column
    df['city'] = df['city'].str.replace(',', '')
    cols = ['period_begin','period_end','period_duration', 'region_type', 'region_type_id', 'table_id',
    'is_seasonally_adjusted', 'city', 'state', 'state_code', 'property_type', 'property_type_id',
    'median_sale_price', 'median_list_price', 'median_ppsf', 'median_list_ppsf', 'homes_sold',
    'inventory', 'months_of_supply', 'median_dom', 'avg_sale_to_list', 'sold_above_list', 'parent_metro_region_metro_code', 
    'last_updated']
    df = df[cols]
    
    df = df.dropna()
    
    #let's change the period_begin and period_end to date time object and extract years and month.
    df['period_begin'] = pd.to_datetime(df['period_begin'])
    df['period_end'] = pd.to_datetime(df['period_end'])
    
    df["period_begin_in_years"] = df['period_begin'].dt.year
    df["period_end_in_years"] = df['period_end'].dt.year

    df["period_begin_in_months"] = df['period_begin'].dt.month
    df["period_end_in_months"] = df['period_end'].dt.month
    
    #let's map the month number to their respective month name.
    month_dict = {
        "period_begin_in_months": {
            1 : "Jan",
             2 :  "Feb",
             3: "Mar",
             4: "Apr",
             5: "May",
             6: "Jun",
             7: "Jul",
             8: "Aug",
             9: "Sep",
             10: "Oct",
             11: "Nov",
             12: "Dec",
        },
        "period_end_in_months": {
            1 : "Jan",
             2 :  "Feb",
             3: "Mar",
             4: "Apr",
             5: "May",
             6: "Jun",
             7: "Jul",
             8: "Aug",
             9: "Sep",
             10: "Oct",
             11: "Nov",
             12: "Dec"
        }}
    df = df.replace(month_dict)
    print('Num of rows:', len(df))
    print('Num of cols:', len(df.columns))
    return df

In [8]:
df = lambda_handler(url_by_city)
df.head()

Num of rows: 4389267
Num of cols: 28


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,city,state,state_code,property_type,property_type_id,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,parent_metro_region_metro_code,last_updated,period_begin_in_years,period_end_in_years,period_begin_in_months,period_end_in_months
0,2012-01-01,2012-01-31,30,place,6,4183,f,Coplay,Pennsylvania,PA,Single Family Residential,6,117000.0,129900.0,97.500000,89.627358,1.0,14.0,14.0,601.0,0.975813,0.000000,10900,2024-11-12 14:29:49,2012,2012,Jan,Jan
1,2022-03-01,2022-03-31,30,place,6,15196,f,Pen Argyl,Pennsylvania,PA,Single Family Residential,6,118750.0,174450.0,90.510671,112.242123,1.0,2.0,2.0,80.0,1.136364,1.000000,10900,2024-11-12 14:29:49,2022,2022,Mar,Mar
2,2018-12-01,2018-12-31,30,place,6,6505,f,Evans,Colorado,CO,All Residential,-1,299000.0,285000.0,134.523810,127.691195,29.0,42.0,1.4,42.0,0.987725,0.137931,24540,2024-11-12 14:29:49,2018,2018,Dec,Dec
4,2012-12-01,2012-12-31,30,place,6,10465,f,Lakewood,New York,NY,All Residential,-1,106250.0,87100.0,73.199745,77.961152,4.0,43.0,10.8,157.0,0.937659,0.000000,27460,2024-11-12 14:29:49,2012,2012,Dec,Dec
5,2022-11-01,2022-11-30,30,place,6,23855,f,Lake Pocotopaug,Connecticut,CT,All Residential,-1,300000.0,285000.0,176.366843,213.434164,3.0,28.0,9.3,91.0,1.019535,1.000000,25540,2024-11-12 14:29:49,2022,2022,Nov,Nov


In [ ]:
#number of years in our dataset
# sorted(df["period_begin_in_years"].unique())

In [ ]:
# download file
# df.to_csv('real_estate_w209_byCity.csv', index=False)